# Reflections for Vehicle Detection Project

### Objective

Detection of other vehicles on the road is a central task in self driving cars. An accurate detection feeds valuable data into the path planning algorithms, which help in staying clear of the other vehicles thereby avoiding collisions. 
In this project we design such a pipeline for detection of other vehicles on the road and track them as they move. 

### Overview of the pipeline

The process of detection and tracking is divided into the following stages:

1) Select a classifier and train it with test images of both cars and not-cars in order to make accurate predictions about new images. In this project we have selected **Linear-SVM** as the classifier of choice. For training the classifier, we extract the following features from the images :

    - Raw pixel color values, binned to an appropriate size.
    - Histogram of the color pixel values.
    - HOG features.

2) Once the classifier is trained, we search for vehicles in the test image. Since the search images are bigger than training images, we need to search for the vehicles in the image. This is implemented by a sliding window algorithm, which generates multiple windows of varying sizes. Once windows are identified, features are extracted from those and fed to the classifier.

3) To eliminate false positives, we enforce a policy of minimum number of windows detecting a positive. Since, it is less likely that a false detection will happen in multiple overlapping windows, this helps then to filter them detections out.

4) A heat map is created, which marks the pixels where positive detection happens.

5) The heatmap serves as an input to a labeling function, which provides the final labels of the vehicles found.

6) **IIR Filtering** : Detections can vary from one input frame to the next. This then produces bounding boxes which jitter from frame to frame, which is not a desired property. To reduce this jitter, following algorithm is adopted :

(a) We accumulate the data of all windows where vehicle was successfully detected over **N (N==5)** frames. After **N**  frames, the oldest detection is dropped.
(b) We keep track of all the detected objects in every frame. If a vehicle is detected, we search in the existing list for a similar detection. By computing the distance between the centroid of the two detection and comparing them to a threshold, we keep track of the objects being detected every frame. These newly detected objects, which maps to an existing object, then are used to filter the centroids, x and y co-ordinates of the bounding boxes as follows :
                * filtered_value = filter_coeff*old_value + (1-filter_coeff)*new_value
                
This helps to smoothen the detection of the vehicles from frame to frame.

(c) If a vehicle is not detected for **N_not_detected (N_not_detected==5)** frames, it is dropped from the list of detected vehicle.
(d) To ensure that the bounding boxes are tight, an **aspect_ratio (==1.5)** is then enforced as well. This makes the x and y lengths proportional.

7) Finally, all the detected objects are drawn on the image with the filtered lengths. 

### Data Exploration

Total training samples for class not_cars ==  8968

Total training samples for class car ==  8792

Below are some samples images from each of the class :

![](exploration/cars.png)
![](exploration/not_cars.png)


### Features

HOG provides a robust method for feature extraction. However, there are many hyper parameters to tune, which makes the performance quite sensitive to the proper selection of all of them. 
To select these hyper parameters, I carried out test runs by varying just one parameter in every run and keeping every other parameter constant.

First, we need to select a color space to work with. I experimented with **RGB**, **HLS** and **YCrCb**.
The below images show that there is no notable difference in the corresponding HOG features in each space. 
This is expected as well since HOG really captures the underlying shape of the object and not really the color variations.
Since, all of the color spaces seem equivalent in terms of HOG, I looked into the performance of the classifier with each one.
**YCrCb** provided with the highest accuracy and hence it was select.

- RGB classifier : 0.9945
- HLS classifier : 0.9955
- YCrCb classifier : 0.998 (although this number seemed to vary from 0.9975 to 0.998 from run to run)

![](exploration/HOG_RGB.png)
![](exploration/HOG_HLS.png)
![](exploration/HOG_YCrCb.png)

With the color space finalized, I started to tune the rest of the parameters.
Performance of these parameters was tested with the test images provided and the once which gave the best result where selected.

pixels_per_cell : Comparison between 16 and 8 was carried out. As shown below, 8 gave much better detection and hence it was selected.
![](exploration/pixels_per_cell_8.png)
![](exploration/pixels_per_cell_16.png)

Cells per blocks : Selection between 2 and 4
![](exploration/cells_per_block_4.png)
![](exploration/cells_per_block_2.png)

Orientations : Selection between 9 and 11
![](exploration/orientation_9.png)
![](exploration/orientation_11.png)

Based on these images, 4 cells per block and 9 orientations were selected.

The class **CClassifier** manages the pipeline for creating and training the classifier. This class provides an public API, **train_classifier()**, for invoking the pipeline for training the classifier. Internally it makes use of 4 private methods to accomplish this task.

The API **train_classifier()** iterates over the list of cars and non cars images and **auguments the data by adding random brightness if desired by the user**. The private method **__extract_features()** extracts the desired color and HOG features from the image and returns the feature vector. These feature vectors are then pre processed by **__data_pre_process()** method, which randomly shuffles the data, divides them into a training/test set and subsequently normalizes the features to zero mean and unit variance. The feature scaler is then saved to the disk in pickled format for later use. Finally, the method **__classifier()** is invoked, which uses the scaled features for training a Linear SVM classifier and saves it to the disk for making predictions later. 

With the selected features and data plus augumented data, my classifier achieved an accuracy of approx **0.998** (although this number seemed to vary from 0.9975 to 0.998 from run to run).

### Sliding window

1) Since the input images from the video are much larger than the size of the cars/non-cars images used for training, we need to divide the input images into smaller windows and search for cars on each individual window. A sliding window algorithm performs this task and is implemented as a private method,**__find_cars()**, of the class **CVehicleDetection**. 
To get the optimal performance, I had to experiment with multiple sizes and scales. Essentially, it is compromise between speed and performance. To get the best performance within a reasonable time, I used 3 window scales (1.2, 1.5, 2.0). These windows are restricted to only the side of the road where the car is driving to avoid detections from the other side. This reduces the number of windows generated and hence improves the speed. Since we only expect to find cars in the second half of the image, the first half is eliminated from the search thereby further improving the run time efficiency.

Following sample images show the various sliding windows which are generated by the algorithm :



![](exploration/sliding_win1.png)
![](exploration/sliding_win2.png)
![](exploration/sliding_win3.png)


2) To improve the reliability of the classifier, I augmented the given data by adding random brightness to the images in the training set. The rationale behind this is since a human eye can still correctly differentiate a car under different lightning conditions, then so should be the case with the classifier if it is sensitive to color values. Since we are you raw color pixels as well as histogram of the colors as the features, the classifier should improve with the additional data. 
Adding this augmented images improved the detection rate in the test set to **0.9975** from **0.98** (although this number seemed to vary from 0.975 to 0.98 from run to run). Some example images of the pipeline running end to end is shown below :

![](exploration/heat_1.png)
![](output_test_images/final_test1.jpg)
![](exploration/heat_2.png)
![](output_test_images/final_test2.jpg)
![](exploration/heat_3.png)
![](output_test_images/final_test3.jpg)
![](exploration/heat_4.png)
![](output_test_images/final_test4.jpg)
![](exploration/heat_5.png)
![](output_test_images/final_test5.jpg)
![](exploration/heat_6.png)
![](output_test_images/final_test6.jpg)

### IIR Filtering

Filtering of false positives is carried in two steps.

1) Within a given frame, we enforce a policy of detecting a minimum number of overlapping windows (**false_detection_threshold**), which would increse confidence in the detection. This policy is enforced by the private mehtod **__apply_threshold()** of the class **CVehicleDetection**. After experimenting with the range of [2,5], I found that detecting minimum of 3 did a good job of filtering out false positive while preserving true detections.

2) The **CVehicleDetection** manages the overall vehicle detection pipeline. It provides an public API, **process_image()**, which can be invoked by outside to trigger the detection of the images. One of the parameters that the constructor for this class takes is iir_filter. This parameter controls if the filtering algorithm, implemented in the private method **__iir_filter()** is invoked or not.

This method aggregates the data of past 5 frames and drops the oldest frame data once 5 frames are detected. The class **CDetectionAttributes** captures the attributes of the detected object like centroid, x1y1, x2y2 etc. In each frame it compares the detected objects with the objects currently in the list by using distance between the 2 centroids as the similarity measure. If a match is found, the existing objects attributes are filtered with tunable a tunable parameter, filter_coeff. 
If an object is not detected for 5 continuous frames, it is dropped from the list. To further ensure that false positives are filtered, any box detected with sides less than 32 pixels is rejected. Also, an object is required to be detected for minimum of **5** frames for it to be plotted. This ensures that false positives which occur from frame to frame are discarded.

### Discussion and Future Work

The biggest challenge I faced in this project was tuning together the sheer number of hyper parameters to get the desired performance. These hyper parameters included: filtering coefficient, amount of overlapping of windows, size and position of the sliding windows, selection of color space etc. It was difficult to segregate these parameters to study them individually and required a lot of experimentation to get to the desired result. A finely tuned pipeline like this may work well on some videos, but it may fail on completely new videos. Therefore, I think other approaches like deep-learning should be implemented as well to compare the performance.

Moreover, since the training mostly consisted of cars, the classifier may not be able to detect other type of vehicles like bike and trucks on the road. I also observed that the pipeline has problems differentiating two cars when they are quite close to one another. This may or may not be a problem depending on how this data is inferred by the algorithms downstream (path planning).